In [34]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
from arcgis.features import GeoSeriesAccessor
import pandas as pd

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)  
# df.spatial.to_featureclass(location=???,sanitize_columns=False)  

# gsa = arcgis.features.GeoSeriesAccessor(df['SHAPE'])  
# df['AREA'] = gsa.area  # KNOW YOUR UNITS

In [35]:
# fill NA values in Spatially enabled dataframes (ignores SHAPE column)
def fill_na_sedf(df_with_shape_column, fill_value=0):
    if 'SHAPE' in list(df_with_shape_column.columns):
        df = df_with_shape_column.copy()
        shape_column = df['SHAPE'].copy()
        del df['SHAPE']
        return df.fillna(fill_value).merge(shape_column,left_index=True, right_index=True, how='inner')
    else:
        raise Exception("Dataframe does not include 'SHAPE' column")

In [36]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [37]:
# read in geometry
taz_832 = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\REMM-Compare-Forecasts\Inputs\TAZ.gdb\TAZ_832")[['TAZID', 'CO_FIPS', 'SHAPE']].copy()
taz_900 = pd.DataFrame.spatial.from_featureclass(r"E:\Tasks\REMM-Compare-Forecasts\Inputs\TAZ.gdb\TAZ_900")[['SA_TAZID', 'CO_FIPS', 'SHAPE']].copy().rename({'SA_TAZID': 'TAZID'}, axis=1)

In [38]:
# read in new
new_832_2019 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\new\taz832_SE_2019.csv").rename({';TAZID': 'TAZID'}, axis=1)
new_900_2019 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\new\taz900_SE_2019.csv").rename({';TAZID': 'TAZID'}, axis=1)
new_832_2050 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\new\taz832_SE_2050.csv").rename({';TAZID': 'TAZID'}, axis=1)
new_900_2050 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\new\taz900_SE_2050.csv").rename({';TAZID': 'TAZID'}, axis=1)

In [39]:
# read in old
old_832_2019 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\old\taz832_SE_2019.csv").rename({';TAZID': 'TAZID'}, axis=1)
old_900_2019 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\old\taz900_SE_2019.csv").rename({';TAZID': 'TAZID'}, axis=1)
old_832_2050 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\old\taz832_SE_2050.csv").rename({';TAZID': 'TAZID'}, axis=1)
old_900_2050 = pd.read_csv(r"E:\Tasks\REMM-Compare-Forecasts\old\taz900_SE_2050.csv").rename({';TAZID': 'TAZID'}, axis=1)

In [40]:
# process new  forecast 
# others I may add later ( RETL	FOOD	MANU	WSLE	OFFI	GVED	HLTH	OTHR)

# 2019
new_832_2019 = new_832_2019[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS', 'RETEMP', 'OTHEMP', 'INDEMP']].copy()
new_832_2019.columns = ['TAZID', 'NEW_TOTHH', 'NEW_TOTEMP', 'NEW_ALLEMP', 'NEW_FM_CONS','NEW_RETEMP', 'NEW_OTHEMP', 'NEW_INDEMP']

new_900_2019 = new_900_2019[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS', 'RETEMP', 'OTHEMP', 'INDEMP']].copy()
new_900_2019.columns = ['TAZID', 'NEW_TOTHH', 'NEW_TOTEMP', 'NEW_ALLEMP', 'NEW_FM_CONS', 'NEW_RETEMP', 'NEW_OTHEMP', 'NEW_INDEMP']

# 2050
new_832_2050 = new_832_2050[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS', 'RETEMP', 'OTHEMP', 'INDEMP']].copy()
new_832_2050.columns = ['TAZID', 'NEW_TOTHH', 'NEW_TOTEMP', 'NEW_ALLEMP', 'NEW_FM_CONS','NEW_RETEMP', 'NEW_OTHEMP', 'NEW_INDEMP']

new_900_2050 = new_900_2050[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS', 'RETEMP', 'OTHEMP', 'INDEMP']].copy()
new_900_2050.columns = ['TAZID', 'NEW_TOTHH', 'NEW_TOTEMP', 'NEW_ALLEMP', 'NEW_FM_CONS', 'NEW_RETEMP', 'NEW_OTHEMP', 'NEW_INDEMP']


In [41]:
# process old forecast

# 2019
old_832_2019 = old_832_2019[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS','RETEMP', 'OTHEMP', 'INDEMP']].copy()
old_832_2019.columns = ['TAZID', 'OLD_TOTHH', 'OLD_TOTEMP', 'OLD_ALLEMP', 'OLD_FM_CONS','OLD_RETEMP', 'OLD_OTHEMP', 'OLD_INDEMP']

old_900_2019 = old_900_2019[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS','RETEMP', 'OTHEMP', 'INDEMP']].copy()
old_900_2019.columns = ['TAZID', 'OLD_TOTHH', 'OLD_TOTEMP', 'OLD_ALLEMP', 'OLD_FM_CONS', 'OLD_RETEMP', 'OLD_OTHEMP', 'OLD_INDEMP']

#2050
old_832_2050 = old_832_2050[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS','RETEMP', 'OTHEMP', 'INDEMP']].copy()
old_832_2050.columns = ['TAZID', 'OLD_TOTHH', 'OLD_TOTEMP', 'OLD_ALLEMP', 'OLD_FM_CONS','OLD_RETEMP', 'OLD_OTHEMP', 'OLD_INDEMP']

old_900_2050 = old_900_2050[['TAZID', 'TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS','RETEMP', 'OTHEMP', 'INDEMP']].copy()
old_900_2050.columns = ['TAZID', 'OLD_TOTHH', 'OLD_TOTEMP', 'OLD_ALLEMP', 'OLD_FM_CONS', 'OLD_RETEMP', 'OLD_OTHEMP', 'OLD_INDEMP']

In [42]:
# join
taz_832_2019_merge = taz_832.merge(old_832_2019, on='TAZID', how='left')
taz_832_2019_merge = taz_832_2019_merge.merge(new_832_2019, on='TAZID', how='left')

taz_832_2050_merge = taz_832.merge(old_832_2050, on='TAZID', how='left')
taz_832_2050_merge = taz_832_2050_merge.merge(new_832_2050, on='TAZID', how='left')

taz_900_2019_merge = taz_900.merge(old_900_2019, on='TAZID', how='left')
taz_900_2019_merge = taz_900_2019_merge.merge(new_900_2019, on='TAZID', how='left')

taz_900_2050_merge = taz_900.merge(old_900_2050, on='TAZID', how='left')
taz_900_2050_merge = taz_900_2050_merge.merge(new_900_2050, on='TAZID', how='left')

In [43]:
# calc difference
fields = ['TOTHH', 'TOTEMP', 'ALLEMP', 'FM_CONS', 'RETEMP', 'OTHEMP', 'INDEMP']
for field in fields:
    taz_832_2019_merge[f'DIFF_{field}'] = taz_832_2019_merge[f'NEW_{field}'] - taz_832_2019_merge[f'OLD_{field}']
    taz_900_2019_merge[f'DIFF_{field}'] = taz_900_2019_merge[f'NEW_{field}'] - taz_900_2019_merge[f'OLD_{field}']
    taz_832_2050_merge[f'DIFF_{field}'] = taz_832_2050_merge[f'NEW_{field}'] - taz_832_2050_merge[f'OLD_{field}']
    taz_900_2050_merge[f'DIFF_{field}'] = taz_900_2050_merge[f'NEW_{field}'] - taz_900_2050_merge[f'OLD_{field}']

In [44]:
# export
taz_832_2019_merge.spatial.to_featureclass(location=os.path.join(gdb2, 'taz832_2019_comparison'),sanitize_columns=False)
taz_900_2019_merge.spatial.to_featureclass(location=os.path.join(gdb2, 'taz900_2019_comparison'),sanitize_columns=False)
taz_832_2050_merge.spatial.to_featureclass(location=os.path.join(gdb2, 'taz832_2050_comparison'),sanitize_columns=False)
taz_900_2050_merge.spatial.to_featureclass(location=os.path.join(gdb2, 'taz900_2050_comparison'),sanitize_columns=False)

'e:\\Tasks\\REMM-Compare-Forecasts\\Outputs\\results.gdb\\taz900_2050_comparison'